In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
from pathlib import Path

# Mount Google Drive

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load features and labels from Google Drive
PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
CHECKPOINT_DIR = PROJECT_ROOT / 'model_checkpoints'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_seed)

# Reshape the features for LSTM input (e.g., (num_samples, seq_len, feature_dim))
seq_len = 1  # This should match your sequence length if it's different
# X_train = X_train.reshape(-1, seq_len, X_train.shape[1])
# X_test = X_test.reshape(-1, seq_len, X_test.shape[1])

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class PARA_CNNLSTM_Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_rate):
        super(PARA_CNNLSTM_Classifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=True)
        self.bn = nn.BatchNorm1d(hidden_dim * 2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=76, kernel_size=(6, 1), padding=(1, 0))
        self.bn1 = nn.BatchNorm2d(76)
        self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
        self.conv2 = nn.Conv2d(in_channels=76, out_channels=111, kernel_size=(4, 1), padding=(1, 0))
        self.bn2 = nn.BatchNorm2d(111)
        self.conv3 = nn.Conv2d(in_channels=111, out_channels=487, kernel_size=(5, 1), padding=(1, 0))
        self.bn3 = nn.BatchNorm2d(487)

        self.flatten_dim = self._get_flatten_dim(input_dim)
        self.fc = nn.Linear(self.flatten_dim, num_classes)
        self.dropout = nn.Dropout(dropout_rate)

    def _get_flatten_dim(self, input_dim):
        h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
        in_lstm = torch.ones(batch_size, 1, input_dim)
        in_cnn = torch.zeros(batch_size, 1, input_dim, 1)
        out_cnn = self.conv1(in_cnn)
        out_cnn = self.bn1(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = self.conv2(out_cnn)
        out_cnn = self.bn2(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = self.conv3(out_cnn)
        out_cnn = self.bn3(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = out_cnn.reshape(batch_size,-1)
        out_lstm, _ = self.lstm(in_lstm, (h0, c0))
        out_lstm = out_lstm[:, -1, :]
        out_lstm = self.bn(out_lstm)
        out_combine = torch.cat([out_cnn, out_lstm], dim=1)
        return out_combine.size(1)

    def forward(self, x):
        # CNN
        in_cnn = x.reshape(-1, 1, X_train.shape[1], 1)
        in_lstm = x.reshape(-1, seq_len, X_train.shape[1])
        out_cnn = self.conv1(in_cnn)
        out_cnn = self.bn1(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = self.conv2(out_cnn)
        out_cnn = self.bn2(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = self.conv3(out_cnn)
        out_cnn = self.bn3(out_cnn)
        out_cnn = F.relu(out_cnn)
        out_cnn = self.pool(out_cnn)
        out_cnn = out_cnn.reshape(out_cnn.size(0),-1)
        # LSTM
        h0 = torch.zeros(self.lstm.num_layers * 2, in_lstm.size(0), self.lstm.hidden_size).to(device)
        c0 = torch.zeros(self.lstm.num_layers * 2, in_lstm.size(0), self.lstm.hidden_size).to(device)
        out_lstm, _ = self.lstm(in_lstm, (h0, c0))
        out_lstm = out_lstm[:, -1, :]
        out_lstm = self.bn(out_lstm)
        out_lstm = self.dropout(out_lstm)
        out_combine = torch.cat([out_cnn, out_lstm], dim=1)
        out = self.fc(out_combine)
        return F.log_softmax(out, dim=1)

# Initialize the improved LSTM model
input_dim = X_train.reshape(-1, seq_len, X_train.shape[1]).shape[2]
hidden_dim = 255
num_layers = 4
dropout_rate = 0.5443207069354133
learning_rate_cnn = 4.579041182623139e-05
learning_rate_fc = 1.0172301244333337e-05
learning_rate = 0.0005533164271008041
num_classes = 2

model = PARA_CNNLSTM_Classifier(input_dim, hidden_dim, num_layers, num_classes, dropout_rate).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam([
    {'params': model.conv1.parameters(), 'lr': learning_rate_cnn},
    {'params': model.bn1.parameters(), 'lr': learning_rate_cnn},
    {'params': model.conv2.parameters(), 'lr': learning_rate_cnn},
    {'params': model.bn2.parameters(), 'lr': learning_rate_cnn},
    {'params': model.conv3.parameters(), 'lr': learning_rate_cnn},
    {'params': model.bn3.parameters(), 'lr': learning_rate_cnn},
    {'params': model.lstm.parameters(), 'lr': learning_rate},
    {'params': model.fc.parameters(), 'lr': learning_rate_fc},  # Assuming you want the same LR as CNN for the final FC layer
    {'params': model.bn.parameters(), 'lr': learning_rate_fc},  # Assuming the same for the batch norm layer after LSTM
], lr=learning_rate_cnn)


# Training loop with early stopping
n_epochs = 30
patience = 5  # Number of epochs to wait for improvement before stopping
best_val_loss = float('inf')
early_stop_counter = 0

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    for data, target in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{n_epochs}", leave=False):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += target.size(0)

    train_loss /= total
    train_losses.append(train_loss)
    train_accuracy = correct / total
    train_accuracies.append(train_accuracy)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc=f"Validating Epoch {epoch+1}/{n_epochs}", leave=False):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

    val_loss /= total
    val_losses.append(val_loss)
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)

    print(f'Epoch {epoch+1}/{n_epochs}')
    print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        model_save_path = CHECKPOINT_DIR / 'best_PARA_CNNLSTM_model_35M.pth'
        torch.save(model.state_dict(), model_save_path)
        print("  Best model saved to Google Drive!")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

# Load the best model
print("Loading the best model from Google Drive...")
model.load_state_dict(torch.load(CHECKPOINT_DIR / 'best_PARA_CNNLSTM_model_35M.pth'))

# Evaluate on the test set
model.eval()
test_loss = 0.0
correct = 0
y_pred_prob = []
y_true = []

print("Evaluating on the test set...")
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Testing", leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        y_pred_prob.extend(output[:, 1].cpu().numpy())
        y_true.extend(target.cpu().numpy())

test_loss /= len(test_loader.dataset)
test_accuracy = correct / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate additional metrics
roc_auc = roc_auc_score(y_true, y_pred_prob)
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)

# Plot Confusion Matrix
y_pred = np.argmax(model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().detach().numpy(), axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()


In [ ]:
import optuna
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch.nn.functional as F
from pathlib import Path

# Mount Google Drive

# Set random seeds for reproducibility
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Load features and labels from Google Drive
PROJECT_ROOT = Path('/content')
EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    PROJECT_ROOT = Path('..').resolve()
    EMBED_DIR = PROJECT_ROOT / 'esm_outputs'
if not EMBED_DIR.exists():
    raise FileNotFoundError(f'Could not find esm_outputs directory at {EMBED_DIR}')

features_path = EMBED_DIR / 'esm_features.npy'
labels_path = EMBED_DIR / 'labels.npy'
CHECKPOINT_DIR = PROJECT_ROOT / 'model_checkpoints'
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
print('Loading embeddings from', features_path)
X = np.load(features_path)
y = np.load(labels_path)


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_seed)

# Reshape the features for LSTM input (e.g., (num_samples, seq_len, feature_dim))
seq_len = 1  # This should match your sequence length if it's different
# X_train = X_train.reshape(-1, 1, X_train.shape[1], 1)
# X_test = X_test.reshape(-1, 1, X_test.shape[1], 1)
# X_train = X_train.reshape(-1, seq_len, X_train.shape[1])
# X_test = X_test.reshape(-1, seq_len, X_test.shape[1])
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
def objective(trial):
    # Suggest hyperparameters
    hidden_dim = 255
    num_layers = 4
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.6)
    learning_rate = 0.0005533164271008041
    learning_rate_cnn = 4.579041182623139e-05
    learning_rate_fc = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    num_filters1 = 76
    num_filters2 = 111
    num_filters3 = 487
    kernel_size1 = 6
    kernel_size2 = 4
    kernel_size3 = 5
    batch_size = trial.suggest_categorical('batch_size', [32, 64])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    class PARA_CNNLSTM_Classifier(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout_rate):
            super(PARA_CNNLSTM_Classifier, self).__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_rate, bidirectional=True)
            self.bn = nn.BatchNorm1d(hidden_dim * 2)
            self.conv1 = nn.Conv2d(in_channels=1, out_channels=num_filters1, kernel_size=(kernel_size1, 1), padding=(1, 0))
            self.bn1 = nn.BatchNorm2d(num_filters1)
            self.pool = nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1))
            self.conv2 = nn.Conv2d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=(kernel_size2, 1), padding=(1, 0))
            self.bn2 = nn.BatchNorm2d(num_filters2)
            self.conv3 = nn.Conv2d(in_channels=num_filters2, out_channels=num_filters3, kernel_size=(kernel_size3, 1), padding=(1, 0))
            self.bn3 = nn.BatchNorm2d(num_filters3)

            # Compute the output dimension size for the final fully connected layer
            # Assuming each dimension is halved by pooling once
            self.flatten_dim = self._get_flatten_dim(input_dim)
            self.fc = nn.Linear(self.flatten_dim, num_classes)
            self.dropout = nn.Dropout(dropout_rate)

        def _get_flatten_dim(self, input_dim):
            h0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
            c0 = torch.zeros(num_layers * 2, batch_size, hidden_dim)
            in_lstm = torch.ones(batch_size, 1, input_dim)
            in_cnn = torch.zeros(batch_size, 1, input_dim, 1)
            out_cnn = self.conv1(in_cnn)
            out_cnn = self.bn1(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = self.conv2(out_cnn)
            out_cnn = self.bn2(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = self.conv3(out_cnn)
            out_cnn = self.bn3(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = out_cnn.reshape(batch_size,-1)
            out_lstm, _ = self.lstm(in_lstm, (h0, c0))
            out_lstm = out_lstm[:, -1, :]
            out_lstm = self.bn(out_lstm)
            out_combine = torch.cat([out_cnn, out_lstm], dim=1)
            return out_combine.size(1)

        def forward(self, x):
            # CNN
            in_cnn = x.reshape(-1, 1, X_train.shape[1], 1)
            in_lstm = x.reshape(-1, seq_len, X_train.shape[1])
            out_cnn = self.conv1(in_cnn)
            out_cnn = self.bn1(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = self.conv2(out_cnn)
            out_cnn = self.bn2(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = self.conv3(out_cnn)
            out_cnn = self.bn3(out_cnn)
            out_cnn = F.relu(out_cnn)
            out_cnn = self.pool(out_cnn)
            out_cnn = out_cnn.reshape(out_cnn.size(0),-1)
            #LSTM
            h0 = torch.zeros(self.lstm.num_layers * 2, in_lstm.size(0), self.lstm.hidden_size).to(device)
            c0 = torch.zeros(self.lstm.num_layers * 2, in_lstm.size(0), self.lstm.hidden_size).to(device)
            # out = out.squeeze(-1)
            # out = out.permute(0, 2, 1)
            out_lstm, _ = self.lstm(in_lstm, (h0, c0))
            out_lstm = out_lstm[:, -1, :]
            out_lstm = self.bn(out_lstm)
            out_lstm = self.dropout(out_lstm)
            out_combine = torch.cat([out_cnn, out_lstm], dim=1)
            out = self.fc(out_combine)
            return F.log_softmax(out, dim=1)

    # Initialize the improved LSTM model
    input_dim = X_train.reshape(-1, seq_len, X_train.shape[1]).shape[2]
    num_classes = 2

    model = PARA_CNNLSTM_Classifier(input_dim, hidden_dim, num_layers, num_classes, dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam([
        {'params': model.conv1.parameters(), 'lr': learning_rate_cnn},
        {'params': model.bn1.parameters(), 'lr': learning_rate_cnn},
        {'params': model.conv2.parameters(), 'lr': learning_rate_cnn},
        {'params': model.bn2.parameters(), 'lr': learning_rate_cnn},
        {'params': model.conv3.parameters(), 'lr': learning_rate_cnn},
        {'params': model.bn3.parameters(), 'lr': learning_rate_cnn},
        {'params': model.lstm.parameters(), 'lr': learning_rate},
        {'params': model.fc.parameters(), 'lr': learning_rate_fc},  # Assuming you want the same LR as CNN for the final FC layer
        {'params': model.bn.parameters(), 'lr': learning_rate_fc},  # Assuming the same for the batch norm layer after LSTM
    ], lr=learning_rate_cnn)


    # Training loop with early stopping
    n_epochs = 30
    patience = 5  # Number of epochs to wait for improvement before stopping
    best_val_loss = float('inf')
    early_stop_counter = 0

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        for data, target in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{n_epochs}", leave=False):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

        train_loss /= total
        train_losses.append(train_loss)
        train_accuracy = correct / total
        train_accuracies.append(train_accuracy)

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc=f"Validating Epoch {epoch+1}/{n_epochs}", leave=False):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item() * data.size(0)
                _, predicted = torch.max(output, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)

        val_loss /= total
        val_losses.append(val_loss)
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

        print(f'Epoch {epoch+1}/{n_epochs}')
        print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            model_save_path = CHECKPOINT_DIR / 'best_PARA_CNNLSTM_model_35M.pth'
            torch.save(model.state_dict(), model_save_path)
            print("  Best model saved to Google Drive!")
        else:
            early_stop_counter += 1
            if early_stop_counter >= 5:
                print("Early stopping triggered")
                break
    # Calculate ROC AUC as the optimization target
    model.eval()
    y_pred_prob = []
    y_true = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            y_pred_prob.extend(output[:, 1].cpu().numpy())
            y_true.extend(target.cpu().numpy())

    roc_auc = roc_auc_score(y_true, y_pred_prob)
    return roc_auc
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
# Step 4: Print the Best Hyperparameters
print('Best hyperparameters:', study.best_params)
print('Best AUC score:', study.best_value)
# Load the best model
print("Loading the best model from Google Drive...")
model.load_state_dict(torch.load(CHECKPOINT_DIR / 'best_PARA_CNNLSTM_model_35M.pth'))

# Evaluate on the test set
model.eval()
test_loss = 0.0
correct = 0
y_pred_prob = []
y_true = []

print("Evaluating on the test set...")
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Testing", leave=False):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        y_pred_prob.extend(output[:, 1].cpu().numpy())
        y_true.extend(target.cpu().numpy())

test_loss /= len(test_loader.dataset)
test_accuracy = correct / len(test_loader.dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate additional metrics
roc_auc = roc_auc_score(y_true, y_pred_prob)
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)

# Plot Confusion Matrix
y_pred = np.argmax(model(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().detach().numpy(), axis=1)
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred))

# ROC Curve and AUC
plt.figure(figsize=(12, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Precision-Recall Curve
plt.figure(figsize=(12, 6))
plt.plot(recall, precision, color='blue', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()
